<a href="https://colab.research.google.com/github/Hernanros/SOTA/blob/master/notebooks/labels/ngram_overlap_calculation_pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install wmd

     |████████████████████████████████| 112kB 2.7MB/s 
  Created wheel for wmd: filename=wmd-1.3.2-cp36-cp36m-linux_x86_64.whl size=629397 sha256=e7f52dec50a1e6425db2700d50bc5a30adc1d4847d8dc0303e34ce83d18503d7
  Stored in directory: /root/.cache/pip/wheels/a6/4d/7a/fc3fdbc12e33c0551301543e4441fb4783c809a30451387ffd
Successfully built wmd


In [4]:
!pip install wordfreq
!git clone https://github.com/brmson/dataset-sts
# !wget  http://nlp.stanford.edu/data/glove.840B.300d.zip

     |████████████████████████████████| 32.8MB 120kB/s 
     |████████████████████████████████| 4.9MB 41.5MB/s 
     |████████████████████████████████| 276kB 41.9MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.3.2-cp36-none-any.whl size=32817238 sha256=952504f69ffa3a22b406cae47ed4b25da31c75af1f836bb21acffbdf84311d63
  Stored in directory: /root/.cache/pip/wheels/8d/ba/84/ba6be76208bd2c2124b6586f7967fb87e9f9fb4b4827e5e2c9
  Created wheel for langcodes: filename=langcodes-2.0.0-cp36-none-any.whl size=5044047 sha256=7416de6f8f2b25e6b7d3c53b2974de25785b666f437834c5603fb75caa42bb94
  Stored in directory: /root/.cache/pip/wheels/c9/11/90/c7bba8118f3674d75e1457537635266a12538cf622a4684bb2
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=862305 sha256=c5cdf6df07c72e286e81fda74bfda0cf282a9ae902bc9d6ef6f2aa12a38abbf7
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built wor

In [5]:
% cd /content/drive/My Drive/NLP_Project

/content/drive/My Drive/NLP_Project


In [12]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from wmd import WMD
import wordfreq
import pickle

In [7]:
zip_ref = zipfile.ZipFile("./glove.840B.300d.zip")
zip_ref.extractall("/tmp")
zip_ref.close()

In [9]:
glove_pd = pd.read_csv('glove_pd.csv',index_col=0)
#glove_pd = pd.read_csv(open('glove.840B.300d.txt'), sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in glove_pd.T.items()}
#glove_pd.to_csv('/content/drive/My Drive/NLP_Project/glove_pd.csv')
# np.savetxt(r'/content/drive/My Drive/NLP_Project/glove.txt', glove_pd.values, fmt='%d', delimiter=' ')

In [17]:
with open('combined_data.pickle', 'rb') as handle:
    df = pickle.load(handle)


In [21]:
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(df.iloc[9,0])
df['text_1_tokens'] =df.text_1.apply(lambda x:tokenizer.tokenize(x)) 
df['text_2_tokens'] =df.text_2.apply(lambda x:tokenizer.tokenize(x)) 

def embed (sent):
  toklist = tokenizer.tokenize(sent)
  embs = []
  freqs = []
  for word in toklist:
    try: 
      embs.append(glove[word.lower()])
      freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))
    except:
      try:
        embs.append(glove[word.strip("'")])
        freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))

      except:
        continue
  return embs, freqs

embed(df.iloc[9,0])
text_1_embeddings = df.text_1.apply(lambda x: embed(x) )
text_2_embeddings = df.text_2.apply(lambda x: embed(x) )

# Word movers distance

In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/content/glove.840B.300d.txt')
tmp_file = get_tmpfile('/content/drive/My Drive/NLP_Project/W2V.txt')

_ = glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
paraphrase['WMD'] = 0
for i in range(len(paraphrase)):
 paraphrase.iloc[i,-1] =  model.wmdistance(paraphrase.iloc[i,0], paraphrase.iloc[i,1])

In [ ]:
#paraphrase = paraphrase.drop(['text_1_tokens','text_2_tokens'],axis = 1)
paraphrase.to_csv('/content/drive/My Drive/NLP_Project/paraphrase.csv')

In [ ]:
paraphrase['WMD']

0      1.382194
1      3.197893
2      2.067205
3      2.630209
4      2.365262
         ...   
993    1.340887
994    2.013165
995    2.395964
996    1.125559
997    2.560768
Name: WMD, Length: 998, dtype: float64

# Word overlap caluculation:
$$ \frac{2 * (S_A \cap S_B)}{S_A + S_B}$$

In [23]:
sent_a = df.iloc[27,0]
sent_b = df.iloc[27,1]

In [24]:
gram = 2

def ngrammer (sent, gram):
  grams = []
  tokenized = tokenizer.tokenize(sent)
  if len(tokenized)>= gram:
    for i in range(len(tokenized)- gram+1):
      grams.append(' '.join(tokenized[i:i+gram]))
  return grams

In [25]:
def gram_overlap (sent_a, sent_b, gram):
  grams_a = set(ngrammer(sent_a,gram))
  grams_b = set(ngrammer(sent_b,gram))
  intersection = np.sum([gram in grams_b for gram in grams_a])
  return 2*intersection/(len(grams_a)+ len(grams_b))




In [27]:
df.head()

,dataset,label,text_1,text_2,bleu_allwords,bleu_withoutstop,glove_allwords,glove_withoutstop,ftext_allwords,ftext_withoutstop,WMD,text_1_tokens,text_2_tokens,1-gram_overlap
0,2012.MSRpar.test.tsv,4.4,The problem likely will mean corrective change...,He said the problem needs to be corrected befo...,0.375739,0.333333,96.20,90.82,77.23,77.39,3,"[The, problem, likely, will, mean, corrective,...","[He, said, the, problem, needs, to, be, correc...",0
1,2012.MSRpar.test.tsv,0.8,The technology-laced Nasdaq Composite Index .I...,The broad Standard & Poor 's 500 Index .SPX in...,0.461538,0.333333,95.66,88.96,87.95,72.39,3,"[The, technology, laced, Nasdaq, Composite, In...","[The, broad, Standard, Poor, s, 500, Index, SP...",0
2,2012.MSRpar.test.tsv,3.6,"`` It 's a huge black eye , '' said publisher ...","`` It 's a huge black eye , '' Arthur Sulzberg...",0.578947,0.500000,97.75,94.94,88.39,72.44,2,"[It, s, a, huge, black, eye, said, publisher, ...","[It, s, a, huge, black, eye, Arthur, Sulzberge...",0
3,2012.MSRpar.test.tsv,3.4,SEC Chairman William Donaldson said there is a...,`` I think there 's a building confidence that...,0.578947,0.444444,98.17,90.51,94.16,75.97,2,"[SEC, Chairman, William, Donaldson, said, ther...","[I, think, there, s, a, building, confidence, ...",0
4,2012.MSRpar.test.tsv,1.4,Vivendi shares closed 1.9 percent at 15.80 eur...,"In New York , Vivendi shares were 1.4 percent ...",0.384615,0.375000,95.77,87.00,90.74,67.48,3,"[Vivendi, shares, closed, 1, 9, percent, at, 1...","[In, New, York, Vivendi, shares, were, 1, 4, p...",0


In [28]:
for n in range(4):
  df[f'{n+1}-gram_overlap'] = 0 
  for i in range(len(df)):
    df.iloc[i,-1] = gram_overlap(df.iloc[i,2],df.iloc[i,3],n+1)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [34]:
df = df.drop(['text_1_tokens','text_2_tokens'],axis = 1)

In [35]:
with open('combined_data.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)